In [1]:
from json import dumps, loads
import time, os
import threading 
import ast
import unittest
import requests
import random
import logging
# chainsapce
from chainspacecontract import transaction_to_solution
# from chainspacecontract.examples.surge import contract as surge_contract
from chainspacecontract.examples import surge
# crypto
from chainspacecontract.examples.utils import setup, key_gen, pack
from chainspaceapi import ChainspaceClient

# Setup logging
logging.basicConfig(level=logging.INFO, filename="execution.log", filemode="a+",
                        format="%(asctime)-15s %(levelname)-8s %(message)s")
logging.info("Starting test_surge_bids")

# Setup variables
r = requests.get('http://10.129.6.52:4999/setup.in')
setup_str = r.text
setup_str = setup_str.split('\n')
NUM_SHARDS = int(setup_str[0])
NUM_REPLICAS = int(setup_str[1])
NUM_CLIENTS = int(setup_str[2])
logging.info("NUM_SHARDS: %s, NUM_REPLICAS: %s, NUM_CLIENTS: %s "%(NUM_SHARDS, NUM_REPLICAS, NUM_CLIENTS))
    
G = setup()[0]


In [2]:
global_client = ChainspaceClient(port=5000)
init_transaction = surge.init()
init_tokens = init_transaction['transaction']['outputs']
global_client.process_transaction(init_transaction)

inputs: ()
reference_inputs: ()
POST http://127.0.0.1:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": [], "methodID": "init", "parameters": [], "outputs": ["{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 1}", "{\"type\": \"InitToken\", \"location\": 1}", "{\"type\": \"InitToken\", \"location\": 1}", "{\"type\": \"InitToken\", \"location\": 1}", "{\"type\": \"InitToken\", \"location\": 1}", "{\"type\": \"InitToken\", \"location\": 1}", "{\"type\": \"InitToken\", \"location\": 1}", "{\"type\": \"InitToken\", \"location\"

<Response [200]>

In [3]:
# Create prosumer clients for locations 
clients = []
base_port=5000
for s in range(0,NUM_SHARDS):
    clients.append([])
    for c in range(0,NUM_CLIENTS):
        clients[s].append(surge.SurgeClient(base_port+s, init_tokens[s*NUM_CLIENTS + c]))

print clients
# c00 = surge.SurgeClient(5000, init_token[0]) # location=0
# Create sreps for locations 0 and 1 which will be clients in location 2
# rep0 = surge.SREPClient(5002, init_token[4]) # location=2

# Elect rep1 as SREP for location 0
# votes = (c00.cast_srep_vote(rep0.pub), c01.cast_srep_vote(rep0.pub))
# rep0.create_srep_client(5000, votes) 

# Elect rep2 as SREP for location 1
# votes = (c10.cast_srep_vote(rep1.pub), c11.cast_srep_vote(rep1.pub))
# rep1.create_srep_client(5001, votes) 


inputs: ('{"type": "InitToken", "location": 0}',)
reference_inputs: ()
POST http://127.0.0.1:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_7db8f6eedbe2c97b84f6465212c38c74be325c462739dab02128e30d5aafa688"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd025fe7c65b23bb228a799216a7a5acb7758590b4e48e6771a03a42855b", "92c71d002bd79bd19fb69d9118c6d9c93a6a21a8d76c912649b595a822013f99fbc71d002b5df6171521bd91175d13b966c6bf228fc5021fc30adc09bc6e046bf1"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 0, \"pub\": \"c7220292cd02c9bd025fe7c65b23bb228a799216a7a5acb7758590b4e48e6771a03a42855b\"}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd025fe7c65b23bb228a799216a7a5acb7758590b4e48e6771a03a42855b\"}", "{\"type\": \"EBToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd025fe7c65b23bb228a799216a7a5acb7758590b4e48e6771a03a42855b\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surg

HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
inputs: ('{"type": "InitToken", "location": 0}',)
reference_inputs: ()
POST http://127.0.0.1:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_c56b53ac9f1a9a8d5594b4a9a7053c6b56dc54311a41a89d0e64f8a2dbd6484b"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd0381e4155bcfd0de8eee50663c765efeefba65ef320fd94fa0f2923695", "92c71d002b6c072bc3284eb93099ba5c5f9712f8317e2f7df36481df9dd420b3cdc71d002b707148b6600e05f24ff9abe103fe2a0e3acf475e0690dd10adcb3732"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 0, \"pub\": \"c7220292cd02c9bd0381e4155bcfd0de8eee50663c765efeefba65ef320fd94fa0f2923695\"}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd0381e4155bcfd0de8eee50663c765efeefba65ef320fd94fa0f2923695\"}", "{\"type\": \"EBToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd0381e4155bcfd0de8eee50663c765efeefba65ef320fd94fa0f2923695\"}"], "return

HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
inputs: ('{"type": "InitToken", "location": 1}',)
reference_inputs: ()
POST http://127.0.0.1:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["1_f010d1f8e51c99721f60b45ad16a8fa698f1950e8302fd6e38911012cc5cf926"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd02bb0627c3758fb592632f279915e88401b878a179f85df3d6069670b8", "92c71d002b140f46c1760f02223463973e2b64eebfd6377d760d7be7f9e855be02c71d002b7e12645aca6e44b0448a2e7d91ce39ca45996f2b78e9007950cfb544"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 1, \"pub\": \"c7220292cd02c9bd02bb0627c3758fb592632f279915e88401b878a179f85df3d6069670b8\"}", "{\"type\": \"VoteSlipToken\", \"location\": 1, \"pub\": \"c7220292cd02c9bd02bb0627c3758fb592632f279915e88401b878a179f85df3d6069670b8\"}", "{\"type\": \"EBToken\", \"location\": 1, \"pub\": \"c7220292cd02c9bd02bb0627c3758fb592632f279915e88401b878a179f85df3d6069670b8\"}"], "return

In [4]:
threads = []
for s in range(0,NUM_SHARDS):
    threads.append([])
    for c in range(0,NUM_CLIENTS):
        client = clients[s][c]
        bid_value = random.randint(0,100)
        threads[s].append(threading.Thread(target=client.submit_bid, args=(random.choice(['EBBuy', 'EBSell']),bid_value)) )

# trep0 = threading.Thread(target=rep0.accept_bids) 


In [5]:
start = time.time()
for s in range(0,NUM_SHARDS):
    for c in range(0,NUM_CLIENTS):
        threads[s][c].start()

for s in range(0,NUM_SHARDS):
    for c in range(0,NUM_CLIENTS):
        threads[s][c].join()

end = time.time()
duration - end-start
logging.info("Execution took "+ str(duration))
logging.info("TPS: %s"%(NUM_CLIENTS/duration))


inputs: ('{"type": "EBToken", "location": 0, "pub": "c7220292cd02c9bd03b98772b61efcf30c319b35d92a072d1d4a87033dc3774161c3985159"}',)
reference_inputs: ()
inputs: ('{"type": "EBToken", "location": 0, "pub": "c7220292cd02c9bd025fe7c65b23bb228a799216a7a5acb7758590b4e48e6771a03a42855b"}',)
inputs: ('{"type": "EBToken", "location": 0, "pub": "c7220292cd02c9bd034d5fde1c6f00fe34929ffaffc9d117f8f0171cc4af6ae0022ae631e9"}',)
reference_inputs: ()
reference_inputs: ()
inputs: ('{"type": "EBToken", "location": 0, "pub": "c7220292cd02c9bd02bdf3b9cd83c1aca75df6e3e806fdf0bd1e06f6801676d7449ea388c9"}',)
reference_inputs: ()
inputs: ('{"type": "EBToken", "location": 0, "pub": "c7220292cd02c9bd021301eb5d245633a2e1d28232faa9d6fb476da354ce9c8e5b731b6d8c"}',)inputs: ('{"type": "EBToken", "location": 0, "pub": "c7220292cd02c9bd02709b7db94d6cd6649b50a80425ab7452ec94ce3f48b3283a6ee6a713"}',)
reference_inputs: ()

reference_inputs: ()
inputs: ('{"type": "EBToken", "location": 0, "pub": "c7220292cd02c9bd0381e41

POST http://127.0.0.1:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["1_6433662126b955fb4dcb93bc0ebf1ac12178b0343a665e85391665cafe7c2657"], "methodID": "submit_bid_proof", "parameters": ["EBBuy", "92c71d002b60059ad283ee2f84923a71eb504e98eb5019cfc8bda8b2ea6b023076c71d002bdaf285dc2acd0e52d1d32b86b4538cebc64edc546fa51abe7f2845a1"], "outputs": ["{\"bid_type\": \"EBBuy\", \"type\": \"BidProof\", \"location\": 1, \"pub\": \"c7220292cd02c9bd0211b65b31bfe0f9456c32b969f5b60a664c5e3c5b070a9c909b3b1d7d\", \"quantity_sig\": \"92c71d002bbe5bf46e2ad6e70824ce8b463d266da9321bc0192e0c0c73de02f39dc71d002b57ecac3a67fbf05400388be33ed7eb7d8de7ffe078f815edf944f4d6\"}", "{\"type\": \"EBToken\", \"location\": 1, \"pub\": \"c7220292cd02c9bd0211b65b31bfe0f9456c32b969f5b60a664c5e3c5b070a9c909b3b1d7d\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"1_6433662126b955fb4dcb93bc0ebf1ac12178b0343a665e85391665cafe7c2657": "{\"type\": \"EBToken

In [6]:
# '{}|{}'.format(50, loads(r1.ebtoken)['pub'])
# global_client.fix_json("{'type':'EBBuy','location':0")
# global_client = ChainspaceClient(port=5001)
# global_client.get_objects({'location':0,'type':'BidAccept'})
# print buy_bids

# print loads(objs[0])['type']
# bid_proofs = rep1.client.get_objects({ 'type':'EBBuy'})
# print init_token1
